In [ ]:
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import h5py
from matplotlib.colors import TwoSlopeNorm

In [ ]:
with h5py.File('mw_tng_sample_catalog.hdf5') as f:
    mw_id = np.array(f['SubfindID'])

In [ ]:
# ------------------ user inputs ------------------
basepath = "/gpfs/gibbs/pi/nagai/IllustrisTNG_data/TNG50-1/output"   # e.g. ".../TNG50-1/output"
snap     = 99                           # z=0
halo_id  = 20                            # <-- FoF halo id you want (integer)
# sub_id = 12345                        # <-- if you prefer a SUBFIND subhalo id
# -------------------------------------------------

# --- cosmology/units (TNG-100/50 use Planck 2015)
h = 0.6774

# --- box size (ckpc/h), for periodic recentering
header  = il.groupcat.loadHeader(basepath, snap)
boxsize = header['BoxSize']            # in ckpc/h

# ------------------ LOADERS ------------------
def load_fof_gas(basepath, snap, halo_id):
    """
    Gas bound to a FoF halo via Subfind catalogs:
    returns positions [kpc], masses [Msun], and a suggested center [kpc].
    """
    # FoF group catalog entry
    grp = il.groupcat.loadSingle(basepath, snap, haloID=halo_id)
    center_ckpch = grp['GroupPos']     # catalog center (ckpc/h)

    # Particle data for that FoF
    gas = il.snapshot.loadHalo(
        basepath, snap, halo_id, partType='gas',
        fields=['Coordinates','Masses']
    )
    coords = gas['Coordinates']        # ckpc/h
    masses = gas['Masses']             # 1e10 Msun/h

    # Convert to kpc and Msun
    coords_kpc  = coords / h
    masses_Msun = masses * (1e10 / h)
    center_kpc  = center_ckpch / h
    box_kpc     = boxsize / h

    # Recenter with periodic wrapping so the halo sits near (0,0,0)
    xyz = coords_kpc - center_kpc
    xyz = (xyz + 0.5*box_kpc) % box_kpc - 0.5*box_kpc

    return xyz, masses_Msun, center_kpc, gas

def load_subhalo_gas(basepath, snap, sub_id):
    """
    Gas bound to a *subhalo* (galaxy) via Subfind catalogs.
    """
    sub = il.groupcat.loadSingle(basepath, snap, subhaloID=sub_id)
    center_ckpch = sub['SubhaloPos']

    gas = il.snapshot.loadSubhalo(
        basepath, snap, sub_id, partType='gas',
        fields=['Coordinates','Masses','Velocit']
    )
    coords = gas['Coordinates']
    masses = gas['Masses']

    coords_kpc  = coords / h
    masses_Msun = masses * (1e10 / h)
    center_kpc  = center_ckpch / h
    box_kpc     = boxsize / h

    xyz = coords_kpc - center_kpc
    xyz = (xyz + 0.5*box_kpc) % box_kpc - 0.5*box_kpc

    return xyz, masses_Msun, center_kpc, gas

In [ ]:
# ------------- choose one of the loaders -------------
#xyz, m_Msun, center_kpc = load_fof_gas(basepath, snap, halo_id)
sub_id = mw_id[6]
xyz, m_Msun, center_kpc = load_subhalo_gas(basepath, snap, sub_id)
# -----------------------------------------------------

# ------------------ 2D surface density map ------------------
# Project along z, bin in x-y
x, y = xyz[:,0], xyz[:,1]

# Choose field of view and resolution
fov_kpc = 400.0                # total width/height of the square in kpc
npix    = 512
edges   = np.linspace(-0.5*fov_kpc, 0.5*fov_kpc, npix+1)

H, xedges, yedges = np.histogram2d(x, y, bins=[edges, edges], weights=m_Msun)
pix_area = (edges[1]-edges[0])**2      # kpc^2
Sigma = H / pix_area                   # Msun / kpc^2

# Plot
fig, ax = plt.subplots(figsize=(6,5))
vmin = max(1e-2, Sigma[Sigma>0].min()) if np.any(Sigma>0) else 1e-2
im = ax.imshow(
    Sigma.T,
    origin="lower",
    extent=[edges[0], edges[-1], edges[0], edges[-1]],
    norm=LogNorm(vmin=vmin, vmax=Sigma.max())
)
cb = plt.colorbar(im, ax=ax, label=r"Gas surface density [$M_\odot\,\mathrm{kpc}^{-2}$]")
ax.set_xlabel("x [kpc]")
ax.set_ylabel("y [kpc]")
ax.set_title(f"TNG50-1 snap {snap} — FoF halo {sub_id} — gas Σ")
plt.tight_layout()
plt.show()

In [ ]:
basePath = '/gpfs/gibbs/pi/nagai/IllustrisTNG_data/TNG50-1/output/'
fields = ['SubhaloMass','SubhaloSFRinRad']
subhalos = il.groupcat.loadSubhalos(basePath,99,fields=fields)

In [ ]:
subhalos.keys()

In [ ]:
mass_msun = subhalos['SubhaloMass'] * 1e10 / 0.704
plt.plot(mass_msun,subhalos['SubhaloSFRinRad'],'.')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Total Mass [$M_\odot$]')
fields = ['Masses']
>>> gas_mass = il.snapshot.loadSubset(basePath,135,'gas',fields=fields)
>>> print np.log10( np.mean(gas_mass,dtype='double')*1e10/0.704 )plt.ylabel('Star Formation Rate [$M_\odot / yr$]')

In [ ]:
basePath = '/gpfs/gibbs/pi/nagai/IllustrisTNG_data/TNG50-1/output/'
fields = ['Masses']
gas_mass = il.snapshot.loadSubset(basePath,99,'gas',fields=fields)
print (np.log10( np.mean(gas_mass,dtype='double')*1e10/0.704 ))